In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('IPL_IMB_data.csv')

In [3]:
data.head(2)

,PLAYER NAME,AGE,COUNTRY,PLAYING ROLE,T-RUNS,T-WKTS,ODI-RUNS-S,ODI-SR-B,ODI-WKTS,ODI-SR-BL,...,HS,AVE,SR-B,SIXERS,RUNS-C,WKTS,AVE-BL,ECON,SR-BL,SOLD PRICE
0,"Abdulla, YA",2,SA,Allrounder,0,0,0,0.00,0,0.0,...,0,0.0,0.0,0,307,15,20.47,8.9,13.93,50000
1,Abdur Razzak,2,BAN,Bowler,214,18,657,71.41,185,37.6,...,0,0.0,0.0,0,29,0,0.00,14.5,0.00,50000


In [4]:
data.shape

(130, 22)

In [5]:
data['AGE']=data['AGE'].astype('object')
data['CAPTAINCY EXP']=data['CAPTAINCY EXP'].astype('object')

In [6]:
data=data.drop('PLAYER NAME',1)

In [7]:
num_data=data.select_dtypes(include=np.number)
cat_data=data.select_dtypes(exclude=np.number)

In [8]:
cat_data.head(2)

,AGE,COUNTRY,PLAYING ROLE,CAPTAINCY EXP
0,2,SA,Allrounder,0
1,2,BAN,Bowler,0


In [9]:
cat_data=pd.get_dummies(cat_data,drop_first=True)

In [10]:
cat_data.head(2)

,AGE_2,AGE_3,COUNTRY_BAN,COUNTRY_ENG,COUNTRY_IND,COUNTRY_NZ,COUNTRY_PAK,COUNTRY_SA,COUNTRY_SL,COUNTRY_WI,COUNTRY_ZIM,PLAYING ROLE_Batsman,PLAYING ROLE_Bowler,PLAYING ROLE_W. Keeper,CAPTAINCY EXP_1
0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
num_data_sc=sc.fit_transform(num_data)
num_data_sc=pd.DataFrame(num_data_sc,columns=num_data.columns)
num_data_sc.head(2)

,T-RUNS,T-WKTS,ODI-RUNS-S,ODI-SR-B,ODI-WKTS,ODI-SR-BL,RUNS-S,HS,AVE,SR-B,SIXERS,RUNS-C,WKTS,AVE-BL,ECON,SR-BL,SOLD PRICE
0,-0.657994,-0.468108,-0.703043,-2.758455,-0.686760,-1.277132,-0.839099,-1.307954,-1.693829,-3.10288,-0.745369,-0.303010,-0.099814,-0.127413,0.547597,-0.226928,-1.162826
1,-0.593006,-0.341460,-0.518927,0.009520,0.983269,0.133821,-0.839099,-1.307954,-1.693829,-3.10288,-0.745369,-0.802864,-0.790019,-1.115257,1.685233,-1.142498,-1.162826


In [12]:
data_full=pd.concat([num_data_sc,cat_data],1)

In [13]:
out=data_full['SOLD PRICE']
inp=data_full.drop('SOLD PRICE',1)

In [14]:
#out.head(10)

In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [16]:
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(inp.values,i) for i in range(inp.shape[1])]
vif['feature']=inp.columns
vif.sort_values('VIF',ascending=False).head(5)

,VIF,feature
15,45.047420,SR-BL
13,44.645799,AVE-BL
11,21.792199,RUNS-C
12,20.391780,WKTS
2,10.972636,ODI-RUNS-S


In [17]:
inp1=inp.drop(['SR-BL','AVE-BL','RUNS-C'],1)
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(inp1.values,i) for i in range(inp1.shape[1])]
vif['feature']=inp1.columns
vif.sort_values('VIF',ascending=False).head(5)

,VIF,feature
2,10.944706,ODI-RUNS-S
6,9.481883,RUNS-S
0,8.395430,T-RUNS
7,8.363663,HS
4,6.855090,ODI-WKTS


In [18]:
import statsmodels.api as sm
inpc=sm.add_constant(inp1)
ols=sm.OLS(out,inpc)
ols_mod=ols.fit()
ols_mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             SOLD PRICE   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.419
Method:                 Least Squares   F-statistic:                     4.322
Date:                Wed, 16 Sep 2020   Prob (F-statistic):           2.95e-08
Time:                        08:53:55   Log-Likelihood:                -133.27
No. Observations:                 130   AIC:                             324.5
Df Residuals:                     101   BIC:                             407.7
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0.2306      0.318      0.725      0.470      -0.401       0.862
T-RUNS                    -0.2921      0.196     -1.489      0.140      -0.681       0.097
T-WKTS                    -0.0933      0.169     -0.552      0.582      -0.428       0.242
ODI-RUNS-S                 0.4246      0.222      1.910      0.059      -0.016       0.866
ODI-SR-B                   0.0017      0.087      0.019      0.985      -0.170       0.174
ODI-WKTS                   0.3344      0.176      1.900      0.060      -0.015       0.684
ODI-SR-BL                 -0.0718      0.085     -0.845      0.400      -0.240       0.097
RUNS-S                     0.2079      0.208      1.000      0.320      -0.204       0.620
HS                        -0.3427      0.194     -1.763      0.081      -0.728       0.043
AVE                        0.3528      0.171      2.063      0.042       0.013       0.692
SR-B                      -0.0781      0.100     -0.785      0.435      -0.276       0.119
SIXERS                     0.1920      0.166      1.158      0.250      -0.137       0.521
WKTS                       0.1409      0.096      1.473      0.144      -0.049       0.331
ECON                      -0.0321      0.091     -0.354      0.724      -0.212       0.148
AGE_2                     -0.5203      0.236     -2.205      0.030      -0.988      -0.052
AGE_3                     -0.7247      0.317     -2.285      0.024      -1.354      -0.096
COUNTRY_BAN               -0.8772      0.914     -0.960      0.339      -2.690       0.935
COUNTRY_ENG                1.4761      0.514      2.874      0.005       0.457       2.495
COUNTRY_IND                0.2869      0.239      1.199      0.233      -0.188       0.762
COUNTRY_NZ                 0.2035      0.361      0.564      0.574      -0.512       0.919
COUNTRY_PAK               -0.2029      0.352     -0.577      0.565      -0.901       0.495
COUNTRY_SA                -0.1403      0.272     -0.516      0.607      -0.679       0.399
COUNTRY_SL                -0.3168      0.312     -1.015      0.312      -0.936       0.302
COUNTRY_WI                -0.6285      0.390     -1.611      0.110      -1.402       0.145
COUNTRY_ZIM               -0.5560      0.881     -0.631      0.529      -2.303       1.191
PLAYING ROLE_Batsman       0.1955      0.251      0.779      0.438      -0.302       0.693
PLAYING ROLE_Bowler        0.0898      0.239      0.375      0.708      -0.385       0.564
PLAYING ROLE_W. Keeper    -0.1570      0.381     -0.412      0.681      -0.914       0.600
CAPTAINCY EXP_1            0.4239      0.247      1.718      0.089      -0.065       0.913
==============================================================================
Omnibus:                        7.050   Durbin-Watson:                   2.058
Prob(Omnibus)